In [1]:
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR

import pandas as pd
import numpy as np


## Load Data

In [2]:
filepath = r'C:\Users\Aaron\Documents\MSc Computer Science\CSE 6242 DVA\Project\Model\tidy_dataset2_v4.csv'
df_original = pd.read_csv(filepath)

In [3]:
# DATA 2

df_original['consumer_goods'] = df_original['Motor_vehicles_and_parts'] + df_original['Furnishings_and_durable_household_equipment'] 
+ df_original['Other_durable_goods'] + df_original['Clothing_and_footwear'] + df_original['Other_nondurable_goods']

df_original['leisure'] = df_original['Recreational_goods_and_vehicles'] + df_original['Recreation_services'] 
+ df_original['Food_services_and_accommodations']

df_original['nutrition'] = df_original['Food_and_beverages_purchased_for_off-premises_consumption']

df_original['mobility'] = df_original['Gasoline_and_other_energy_goods'] + df_original['Transportation_services']

df_original['housing'] = df_original['Housing_and_utilities']

df_original['services'] = df_original['Health_care'] + df_original['Financial_services_and_insurance'] + df_original['Other_services']

# drop original variables

df = df_original[['State', 'Year', 'CO2', 'consumer_goods', 'leisure', 'nutrition', 'mobility', 'housing', 'services']]

df.head()

,State,Year,CO2,consumer_goods,leisure,nutrition,mobility,housing,services
0,Alabama,1997,8.812122e+07,7054.8,4685.5,7747.3,4758.1,13167.0,23703.9
1,Alabama,1998,8.502097e+07,7562.6,4865.6,7866.4,4597.8,13826.6,24828.4
2,Alabama,1999,8.784891e+07,8201.6,5141.0,8166.4,4975.7,14515.5,25752.1
3,Alabama,2000,9.621875e+07,8358.4,5348.3,8335.8,5779.5,15473.6,27456.8
4,Alabama,2001,1.038840e+08,8629.9,5378.3,8794.1,5540.9,16588.3,28581.8


## Run VAR Model for each state

In [4]:
models =[]
state_predicted_CO2 = []
for state, state_df in df.groupby('State'):
    temp_df = state_df.set_index('Year')
    
    train_df = temp_df[:-5]
    test_df = temp_df[-5:]
    
    models.append(VAR(train_df[train_df.columns[~train_df.columns.isin(['State', 'Year'])]].diff()[1:]))
    

In [37]:
sorted_order=models[5].select_order()
print(sorted_order.summary())

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0       103.2       103.5   6.302e+44       103.2
1      100.7*      103.4*  8.243e+43*      101.0*
-------------------------------------------------


In [26]:
var_models=[]
predict=[]
predictions=[]
mdape=[]

n_forecast = 5


for state, state_df in df.groupby('State'):
    train_df = state_df[:-5]
    train_df = train_df.set_index('Year')
    test_df = state_df[-5:]

    var_model = VARMAX(train_df[train_df.columns[~train_df.columns.isin(['State'])]], order=(1,0),enforce_stationarity= True)
    var_models.append(var_model.fit(disp=False))
    
    predict.append(var_models[-1].get_prediction(start=len(train_df),end=len(train_df) + n_forecast-1))

    predictions.append(predict[-1].predicted_mean)
    
    mdape.append(np.median((np.abs(np.subtract(test_df['CO2'].reset_index(), predictions[-1]['CO2'].reset_index())/ test_df['CO2'].reset_index()))) * 100)

In [36]:
dataset = pd.DataFrame({'State': np.unique(df['State']), 'mape': mdape})
dataset
dataset.to_csv('VAR_MDAPE.csv')

In [38]:
#Mean MDAPE for US
np.mean(mdape)

68.84029767947061